# 标题

我下面有这些模型：

传统机器学习模型: Logitics Regression（线性）, SVM（线性+kernel）, Random Rorest, XGboost（后面这两决策树）
深度学习模型：CNN, LSTM, Vision Transformer

接下来需要导出模型：

```
我这边用TensorFlow或keras训练了以下<>模型，做的是MNIST分类任务
<复制代码>

现在我想把这个tf/keras模型导出为<xxx>格式，请告诉我应该怎么导出
```

## LSTM

In [2]:
# ~ 20 mins -> 0.5-0.9s 
import threading
import time
import subprocess
import os
import psutil
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input
import tempfile

# 全局变量用于存储子线程的输出
thread_output = {}

# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 数据预处理和reshape
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# LSTM 需要3D输入，将数据reshape为(samples, time steps, features)
time_steps = 28  # 因为MNIST图片大小为28x28
features = 28    # 每个时间步的特征数
X_train = X_train.reshape(-1, time_steps, features)
X_test = X_test.reshape(-1, time_steps, features)

# 将标签进行 one-hot 编码
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 构建更大的 LSTM 模型
model = Sequential([
    Input(shape=(time_steps, features)),
    LSTM(units=256, input_shape=(time_steps, features), return_sequences=True),  # 第一层LSTM，返回序列以叠加另一层LSTM
    LSTM(units=256),  # 第二层LSTM
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 自定义批次生成器
def batch_generator(X, y, batch_size):
    while True:
        for start in range(0, len(X), batch_size):
            end = min(start + batch_size, len(X))
            yield X[start:end], y[start:end]

batch_size = 128
train_gen = batch_generator(X_train, y_train, batch_size)

# 监控资源使用情况
def monitor_resources():
    memory_info = psutil.virtual_memory()
    print(f"Memory usage: {memory_info.percent}%")
    cpu_usage = psutil.cpu_percent(interval=1)
    print(f"CPU usage: {cpu_usage}%")

# 训练模型并监控资源使用情况
steps_per_epoch = len(X_train) // batch_size
model.fit(train_gen, epochs=1, steps_per_epoch=steps_per_epoch, verbose=2, 
          callbacks=[tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: monitor_resources())])

# 评估模型
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {score[1]}')
# 省略后续监控保存模型的代码，可以沿用上述已有的代码

2024-07-08 22:42:13.061067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

## XGboost

In [ ]:
import numpy as np
import xgboost as xgb
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 将图像数据从 3D 重塑为 2D (每个图像为一行，像素为列)
X_train_2d = X_train.reshape(-1, 28*28)
X_test_2d = X_test.reshape(-1, 28*28)

# 将数据类型转换为 float32
X_train_2d = X_train_2d.astype('float32') / 255
X_test_2d = X_test_2d.astype('float32') / 255

# XGBoost 对象
model = xgb.XGBClassifier(
    objective='multi:softmax',  # 目标函数
    num_class=10,               # 类别数，与 MNIST 的标签数量相同
    n_estimators=100,           # 树的个数
    max_depth=6,                # 树的深度
    learning_rate=0.1           # 学习速率
)

# 训练模型
model.fit(X_train_2d, y_train)

# 预测测试集
y_pred = model.predict(X_test_2d)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy: {:.2f}%".format(accuracy * 100))

Test accuracy: 97.02%


## SVM

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist

# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 将图像数据从 3D 重塑为 2D (每个图像为一行，像素为列)
X_train_2d = X_train.reshape(-1, 28*28)
X_test_2d = X_test.reshape(-1, 28*28)

# 将数据类型转换为 float32 并进行归一化处理
X_train_2d = X_train_2d.astype('float32') / 255
X_test_2d = X_test_2d.astype('float32') / 255

# 创建 SVM 分类器
# 因为 MNIST 数据集较大，使用线性核通常是一个不错的选择，同时考虑到效率，可以使用 LinearSVC 或者使用 SVC 时设置 kernel='linear'
clf = svm.SVC(kernel='linear')

# 训练模型
clf.fit(X_train_2d, y_train)

# 预测测试集
y_pred = clf.predict(X_test_2d)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy * 100:.2f}%')


Test accuracy: 94.04%


## logistics regression

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist

# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 将图像数据从 3D 重塑为 2D (每个图像为一行，像素为列)
X_train_2d = X_train.reshape(-1, 28*28)
X_test_2d = X_test.reshape(-1, 28*28)

# 将数据类型转换为 float32 并进行归一化处理
X_train_2d = X_train_2d.astype('float32') / 255
X_test_2d = X_test_2d.astype('float32') / 255

# 创建逻辑回归分类器
# 使用 lbfgs 求解器，因为它支持多分类，并且适用于较大的数据集
clf = LogisticRegression(solver='lbfgs', max_iter=200, multi_class='multinomial')
# clf = LogisticRegression(solver='saga', max_iter=200, multi_class='multinomial')

# 训练模型
clf.fit(X_train_2d, y_train)

# 预测测试集
y_pred = clf.predict(X_test_2d)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy * 100:.2f}%')


Test accuracy: 92.64%


/opt/anaconda3/envs/env_name/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Random Forest

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist

# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 将图像数据从 3D 重塑为 2D (每个图像为一行，像素为列)
X_train_2d = X_train.reshape(-1, 28*28)
X_test_2d = X_test.reshape(-1, 28*28)

# 将数据类型转换为 float32 并进行归一化处理
X_train_2d = X_train_2d.astype('float32') / 255
X_test_2d = X_test_2d.astype('float32') / 255

# 创建随机森林分类器
# n_estimators 表示使用的决策树数量，更多的树会提高模型的性能和稳定性，但也会增加计算成本
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练模型
clf.fit(X_train_2d, y_train)

# 预测测试集
y_pred = clf.predict(X_test_2d)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy * 100:.2f}%')


Test accuracy: 97.04%


## CNN

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# 加载 MNIST 数据集
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 调整输入数据的形状和范围
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# 将标签转换为 one-hot 编码
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 创建 CNN 模型
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, batch_size=128, epochs=2, verbose=1, validation_data=(X_test, y_test))

# 评估模型
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('./mnist_cnn_model_new.h5')

2024-07-22 20:26:46.172831: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-22 20:26:50.051044: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2
469/469 [==============================] - 10s 19ms/step - loss: 0.2951 - accuracy: 0.9103 - val_loss: 0.0576 - val_accuracy: 0.9816
Epoch 2/2
469/469 [==============================] - 9s 19ms/step - loss: 0.0915 - accuracy: 0.9726 - val_loss: 0.0409 - val_accuracy: 0.9861
Test loss: 0.0408916175365448
Test accuracy: 0.9861000180244446


In [ ]:
import threading
import time
import subprocess
import os
import psutil
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import tempfile

thread_output = {}

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        # time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event):
    print("Saving model...")
    start_time = time.time()
    # 保存 Keras 模型为 HDF5 格式
    
    for i in range(0, 50):
        model.save("mnist_cnn_model.h5")
    
    
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save TensorFlow SavedModel: {duration} seconds')
    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                # time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    # process.wait(timeout=5)  # 尝试在5秒内等待进程结束
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()

# model = tf.keras.models.load_model("mnist_cnn_model.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event,))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))

thread3.start()
thread1.start()
thread2.start()


# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
print(thread_output.get('powermetrics', 'No output captured'))
# print('CPU usage during saving:', thread_output.get('cpu_usage', 'No CPU usage captured'))
# print('GPU usage during saving:', thread_output.get('gpu_usage', 'No GPU usage captured'))

In [2]:
import threading
import time
import subprocess
import os
import psutil
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import tempfile

thread_output = {}

# 定义监控保存模型时的资源使用率的线程函数
# 跑完上面的模型，测试以下代码
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None
        # time.sleep(0.1)

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event):
    print("Saving model...")
    start_time = time.time()
    # 保存 Keras 模型为 HDF5 格式
    
    # time.sleep(0.5)
    # model.save("mnist_lstm_model.h5")
    
    for i in range(0, 50):
        # 保存模型的操作
        model.save("mnist_cnn_model.h5")
    
    
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save TensorFlow SavedModel: {duration} seconds')
    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
                # time.sleep(0.1)
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    # process.wait(timeout=5)  # 尝试在5秒内等待进程结束
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

stop_event = threading.Event()

model = tf.keras.models.load_model("mnist_cnn_model_new.h5")

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event,))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))

thread3.start()
thread1.start()
thread2.start()


# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
print(thread_output.get('powermetrics', 'No output captured'))

Saving model...
Subprocess started.
Time taken to save TensorFlow SavedModel: 1.9674737453460693 seconds
Subprocess finished.
Resource monitoring finished.
Machine model: MacBookPro18,1
OS version: 23F79
Boot arguments: 
Boot time: Mon Jul  8 21:39:45 2024



*** Sampled system activity (Mon Jul 22 20:28:10 2024 +0200) (16.21ms elapsed) ***


**** Processor usage ****

E-Cluster Online: 100%
E-Cluster HW active frequency: 2015 MHz
E-Cluster HW active residency:  70.14% (600 MHz:   0% 972 MHz: 2.9% 1332 MHz:   0% 1704 MHz: 4.8% 2064 MHz:  92%)
E-Cluster idle residency:  29.86%
CPU 0 frequency: 2013 MHz
CPU 0 active residency:  48.27% (600 MHz:   0% 972 MHz: 1.6% 1332 MHz:   0% 1704 MHz: 1.9% 2064 MHz:  45%)
CPU 0 idle residency:  51.73%
CPU 1 frequency: 2041 MHz
CPU 1 active residency:  47.23% (600 MHz:   0% 972 MHz: .51% 1332 MHz:   0% 1704 MHz: 1.5% 2064 MHz:  45%)
CPU 1 idle residency:  52.77%

P0-Cluster Online: 100%
P0-Cluster HW active frequency: 3067 MHz
P0-Cluster HW active resi

## ViT
Vision Transformer

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import numpy as np

# Vision Transformer 相关参数
img_size = 28
patch_size = 7
num_patches = (img_size // patch_size) ** 2
proj_dim = 64
num_heads = 4
transformer_layers = 8
mlp_dim = 128
num_classes = 10

# 加载 MNIST 数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.astype("float32") / 255.0, x_test.astype("float32") / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# 创建 Patches Layer
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

# 创建 Patch Encoder Layer
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, proj_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.proj = layers.Dense(units=proj_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=proj_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.proj(patch) + self.position_embedding(positions)
        return encoded

# 创建 Vision Transformer 模型
def create_vit_classifier():
    inputs = layers.Input(shape=(img_size, img_size, 1))
    patches = Patches(patch_size)(inputs)
    encoded_patches = PatchEncoder(num_patches, proj_dim)(patches)

    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=proj_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = layers.Dense(units=mlp_dim, activation=tf.nn.gelu)(x3)
        x3 = layers.Dense(units=proj_dim)(x3)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dense(units=mlp_dim, activation=tf.nn.gelu)(representation)
    representation = layers.Dropout(0.1)(representation)

    # Add MLP head.
    logits = layers.Dense(num_classes)(representation)
    return models.Model(inputs=inputs, outputs=logits)

# 编译并训练模型
model = create_vit_classifier()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=5,
    validation_split=0.1,
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

/opt/anaconda3/envs/env_name/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/env_name/lib/python3.8/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_patch16_224_in21k to current vit_base_patch16_224.augreg_in21k.
  model = create_fn(


KeyboardInterrupt: 

In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 加载 IMDB 数据集
max_features = 20000  # 使用的单词数量
maxlen = 100  # 每条评论的最大长度

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# 将序列转换为文本
word_index = imdb.get_word_index()
index_word = {v: k for k, v in word_index.items()}

def sequences_to_texts(sequences):
    return [' '.join([index_word.get(i - 3, '?') for i in seq]) for seq in sequences]

X_train_text = sequences_to_texts(X_train)
X_test_text = sequences_to_texts(X_test)

# 使用TF-IDF向量化文本数据
vectorizer = TfidfVectorizer(max_features=max_features)
X_train_tfidf = vectorizer.fit_transform(X_train_text)
X_test_tfidf = vectorizer.transform(X_test_text)

# 创建 SVM 分类器
clf = svm.SVC(kernel='linear')

# 训练模型
clf.fit(X_train_tfidf, y_train)

# 预测测试集
y_pred = clf.predict(X_test_tfidf)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy * 100:.2f}%')